# Improved Glove Rnn

In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, GlobalMaxPool1D, SimpleRNN, GRU
from keras.models import Model
# from keras import initializers, regularizers, constraints, optimizers, layers

In [2]:
# quick and dirty way to change the current working directory to root (/toxic-comment-classification)
# you should run this at least once just to be certain
from os import chdir, path, getcwd
if getcwd().endswith("src"):
    chdir(path.pardir)
if path.isfile("checkcwd"):
    print("Success")
else:
    raise Exception("Something went wrong. cwd=" + getcwd())
root_path = os.getcwd()

Success


In [3]:
path = 'kaggle/input/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
clean_data_path = 'clean_data/'
EMBEDDING_FILE=f'{path}glove_embeddings/glove.6B.300d.txt'
TRAIN_DATA_FILE=f'{path}{comp}train.csv.zip'
TEST_DATA_FILE=f'{path}{comp}test.csv.zip'
CLEAN_TRAIN_DATA_FILE=f'{clean_data_path}data_train_cleaned_vanilla2.txt'
CLEAN_TEST_DATA_FILE=f'{clean_data_path}data_test_cleaned_vanilla2.txt'
SAMPLE_SUBMISSION=f'{path}{comp}sample_submission.csv.zip'

Embedding parameters

In [4]:
embed_size = 300 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

Read data

In [5]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

def read_from_file(filename):
    with open(filename, 'r') as f:
        return f.read().splitlines()
    
list_sentences_train = read_from_file(CLEAN_TRAIN_DATA_FILE)
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = read_from_file(CLEAN_TEST_DATA_FILE)

Standard keras preprocessing, to turn each comment into a list of word indexes of equal length (with truncation or padding as needed).

In [6]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

In [7]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

Use these vectors to create our embedding matrix, with random initialization for words that aren't in GloVe. We'll use the same mean and stdev of embeddings the GloVe has when generating the random init.

In [8]:
all_embs = np.stack(list(embeddings_index.values()))
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(-0.0039050116, 0.38177028)

In [9]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

Create Model

In [10]:
from keras.metrics import AUC
def get_model(layertype='RNN', use_dropout=False, dropout_rate=0.1):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    if layertype == 'RNN' and not use_dropout:
        x = Bidirectional(SimpleRNN(60, return_sequences=True))(x)
    elif layertype == 'LSTM' and not use_dropout:
        x = Bidirectional(LSTM(60, return_sequences=True))(x)
    elif layertype == 'GRU' and not use_dropout:
        x = Bidirectional(GRU(60, return_sequences=True))(x)
    elif layertype == 'RNN' and use_dropout:
        x = Bidirectional(SimpleRNN(60, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate))(x)
    elif layertype == 'LSTM' and use_dropout:
        x = Bidirectional(LSTM(60, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate))(x)
    elif layertype == 'GRU' and use_dropout:
        x = Bidirectional(GRU(60, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[AUC(name='auc')])
    return model

Create Validation Set

In [11]:
# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(X_t, y, test_size=0.1, random_state=7)

Training and eval

In [12]:
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

# class CustomCallback(Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []
#         self.aucs = []

#     def on_batch_end(self, batch, logs={}):
#         if batch % 500 == 0:
#             loss, auc = self.model.evaluate(X_val, y_val, verbose=0)
#             self.losses.append(loss)
#             self.aucs.append(auc)
#             print(f'\nEvaluation at batch {batch}: Loss = {loss}, AUC = {auc}\n')

# # Instantiate the custom callback
# custom_callback = CustomCallback()
checkpoint_path = 'model_checkpoint/'
rnn_checkpoint = ModelCheckpoint(checkpoint_path + 'rnn.keras', monitor='val_auc', mode='max', save_best_only=True, verbose=1)
lstm_checkpoint = ModelCheckpoint(checkpoint_path + 'lstm.keras', monitor='val_auc', mode='max', save_best_only=True, verbose=1)
gru_checkpoint = ModelCheckpoint(checkpoint_path + 'gru.keras', monitor='val_auc', mode='max', save_best_only=True, verbose=1)
# early_stopping = EarlyStopping(monitor='loss', min_delta=0.0005, restore_best_weights=True)

In [13]:
# import matplotlib.pyplot as plt

# def plot_loss(callback):
#     plt.figure(figsize=(12,6))
#     plt.plot(callback.losses)
#     plt.title('model loss')
#     plt.ylabel('loss')
#     plt.xlabel('500 batch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()
    
# def plot_auc(callback):
#     plt.figure(figsize=(12,6))
#     plt.plot(callback.aucs)
#     plt.title('model AUC')
#     plt.ylabel('AUC')
#     plt.xlabel('500 batch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()

# def plot_loss_auc(history):
#     plot_loss(history)
#     plot_auc(history)

In [14]:
import gc
gc.collect()

0

In [15]:
model = get_model('RNN', True, 0.15)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 300)       │     6,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 120)       │        43,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 120)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 50)             │         6,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           306 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,049,676 (23.08 MB)

 Trainable params: 6,049,676 (23.08 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
batch_size = 64
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, callbacks=[rnn_checkpoint], validation_split=0.1)

Epoch 1/3
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - auc: 0.9179 - loss: 0.0916
Epoch 1: val_auc improved from -inf to 0.98010, saving model to model_checkpoint/rnn.keras
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 75s 32ms/step - auc: 0.9179 - loss: 0.0916 - val_auc: 0.9801 - val_loss: 0.0488
Epoch 2/3
2243/2244 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - auc: 0.9836 - loss: 0.0468
Epoch 2: val_auc improved from 0.98010 to 0.98358, saving model to model_checkpoint/rnn.keras
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 76s 34ms/step - auc: 0.9835 - loss: 0.0468 - val_auc: 0.9836 - val_loss: 0.0463
Epoch 3/3
2243/2244 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - auc: 0.9878 - loss: 0.0404
Epoch 3: val_auc did not improve from 0.98358
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 75s 34ms/step - auc: 0.9878 - loss: 0.0404 - val_auc: 0.9798 - val_loss: 0.0471


And finally, get predictions for the test set and prepare a submission CSV:

In [17]:
model.load_weights(checkpoint_path + 'rnn.keras')
sample_submission = pd.read_csv(SAMPLE_SUBMISSION)
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission[list_classes] = y_test
sample_submission.to_csv(root_path + '/kaggle/working/' + 'rnn_glove_submission.csv', index=False)

150/150 ━━━━━━━━━━━━━━━━━━━━ 12s 76ms/step


Build Lstm model

In [18]:
del model
model = get_model('LSTM', True, 0.15)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 100, 300)       │     6,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 100, 120)       │       173,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 120)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50)             │         6,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           306 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,179,636 (23.57 MB)

 Trainable params: 6,179,636 (23.57 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, callbacks=[lstm_checkpoint], validation_split=0.1)

Epoch 1/3
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - auc: 0.9184 - loss: 0.0960
Epoch 1: val_auc improved from -inf to 0.98262, saving model to model_checkpoint/lstm.keras
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 124s 54ms/step - auc: 0.9184 - loss: 0.0960 - val_auc: 0.9826 - val_loss: 0.0456
Epoch 2/3
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - auc: 0.9853 - loss: 0.0428
Epoch 2: val_auc did not improve from 0.98262
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 124s 55ms/step - auc: 0.9853 - loss: 0.0428 - val_auc: 0.9815 - val_loss: 0.0451
Epoch 3/3
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - auc: 0.9891 - loss: 0.0374
Epoch 3: val_auc did not improve from 0.98262
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 123s 55ms/step - auc: 0.9891 - loss: 0.0374 - val_auc: 0.9798 - val_loss: 0.0454


In [20]:
model.load_weights(checkpoint_path + 'lstm.keras')
sample_submission = pd.read_csv(SAMPLE_SUBMISSION)
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission[list_classes] = y_test
sample_submission.to_csv(root_path + '/kaggle/working/' + 'lstm_glove_submission.csv', index=False)

150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 162ms/step


Build GRU model

In [21]:
del model
model = get_model('GRU', True, 0.1)
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 100, 300)       │     6,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 100, 120)       │       130,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ (None, 120)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 50)             │         6,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │           306 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,136,676 (23.41 MB)

 Trainable params: 6,136,676 (23.41 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, callbacks=[gru_checkpoint], validation_split=0.1)

Epoch 1/3
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - auc: 0.9270 - loss: 0.0943
Epoch 1: val_auc improved from -inf to 0.98445, saving model to model_checkpoint/gru.keras
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 135s 59ms/step - auc: 0.9270 - loss: 0.0943 - val_auc: 0.9844 - val_loss: 0.0491
Epoch 2/3
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - auc: 0.9839 - loss: 0.0589
Epoch 2: val_auc improved from 0.98445 to 0.98467, saving model to model_checkpoint/gru.keras
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 131s 58ms/step - auc: 0.9839 - loss: 0.0589 - val_auc: 0.9847 - val_loss: 0.0466
Epoch 3/3
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - auc: 0.9874 - loss: 0.0712
Epoch 3: val_auc did not improve from 0.98467
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 131s 58ms/step - auc: 0.9874 - loss: 0.0712 - val_auc: 0.9844 - val_loss: 0.0470


In [23]:
model.load_weights(checkpoint_path + 'gru.keras')
sample_submission = pd.read_csv(SAMPLE_SUBMISSION)
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission[list_classes] = y_test
sample_submission.to_csv(root_path + '/kaggle/working/' + 'gru_glove_submission.csv', index=False)

150/150 ━━━━━━━━━━━━━━━━━━━━ 21s 139ms/step


In [24]:
lstm_submission = pd.read_csv(root_path + '/kaggle/working/' + 'lstm_glove_submission.csv')
vanilla_submission = pd.read_csv(root_path + '/kaggle/working/' + 'vanilla_submission.csv')
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
ensemble_submission = lstm_submission.copy()
ensemble_submission[label_cols] = (vanilla_submission[label_cols] + lstm_submission[label_cols]) / 2
ensemble_submission.to_csv(root_path + '/kaggle/working/' + 'nbsvm_lstm_ensemble_submission.csv', index=False)